In [1]:
import torch
from torch import nn

In [2]:
def corr2d(X, K):
    h, w = K.shape

    # (nh-kh+1)*(nw-kw+1).
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))

    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()

    return Y


In [3]:
X = torch.tensor(
    [[0, 1, 2],
     [3, 4, 5],
     [6, 7, 8]]
)

K = torch.tensor(
    [[0, 1],
     [2, 3]]
)

corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [5]:
X = torch.ones((6, 8))
X

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [6]:
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
K = torch.tensor([[1, -1]])

In [8]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [9]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(100):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()

    conv2d.weight.data[:] -= lr * conv2d.weight.grad

    if (i + 1) % 10 == 0:
        print(f'epoch{i + 1},loss {l.sum():.3f}')

epoch10,loss 0.056
epoch20,loss 0.001
epoch30,loss 0.000
epoch40,loss 0.000
epoch50,loss 0.000
epoch60,loss 0.000
epoch70,loss 0.000
epoch80,loss 0.000
epoch90,loss 0.000
epoch100,loss 0.000


In [10]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0000, -1.0000]])

In [11]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略批量大小和通道
    return Y.reshape(Y.shape[2:])


# conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1)

X = torch.randn((8, 8))
# comp_conv2d(conv2d,X).shape

In [12]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [13]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [14]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K))


X = torch.randn((2, 3, 3))
X

tensor([[[-1.2957,  0.8891,  0.0235],
         [-0.5711,  2.0256, -0.1422],
         [-0.2474,  1.7259,  0.8036]],

        [[-0.0791, -0.6098,  0.3627],
         [-0.2621,  1.9796, -0.8619],
         [-0.8592,  1.9132, -0.0915]]])

In [15]:
K = torch.randn((2, 2, 2))
K

tensor([[[-0.2811,  1.7990],
         [-0.1990, -1.3976]],

        [[ 0.6460,  2.3017],
         [ 0.1099,  1.1557]]])

In [16]:
corr2d_multi_in(X, K)

tensor([[ 0.0507, -0.7495],
        [ 7.9454, -2.8922]])

In [17]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [18]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [19]:
corr2d_multi_in_out(X, K)

tensor([[[ 0.0507, -0.7495],
         [ 7.9454, -2.8922]],

        [[ 2.1272,  2.9171],
         [13.6500,  4.4601]],

        [[ 4.2037,  6.5838],
         [19.3546, 11.8125]]])

In [22]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y1

tensor([[[-1.8804, -0.4581, -1.2173],
         [ 1.4326, -0.5600, -0.5311],
         [-1.0435, -0.3202, -0.2829]],

        [[ 2.2562, -2.0430, -0.5775],
         [ 0.1466,  1.1836,  0.2400],
         [-0.8999, -0.2738,  1.0978]]])

In [23]:
Y2 = corr2d_multi_in_out(X, K)
Y2

tensor([[[-1.8804, -0.4581, -1.2173],
         [ 1.4326, -0.5600, -0.5311],
         [-1.0435, -0.3202, -0.2829]],

        [[ 2.2562, -2.0430, -0.5775],
         [ 0.1466,  1.1836,  0.2400],
         [-0.8999, -0.2738,  1.0978]]])